# Common requirements

In [1]:
# ! pip install clickhouse_connect
# ! pip install pendulum
# ! pip install pydantic

In [1]:
import requests
import pandas as pd  # DataFrame
import math
import pendulum  # more convinient datetime
import time
import os
import json


import clickhouse_connect
from clickhouse_connect.driver.tools import insert_file

**Авторизационный токен будет действителен в течение года**. Необходимо обновить OAuth-токен **14.04.2025**

Процедура получения нового токена - https://appmetrica.yandex.ru/docs/ru/mobile-api/intro/authorization

Обновление токенов - https://yandex.ru/dev/id/doc/ru/tokens/refresh-client

Запрос токена (общий) - "https://oauth.yandex.ru/authorize?response_type=token&client_id=<идентификатор приложения>"


# AppMetrica

In [2]:
key = os.getenv("API_KEY")

У этого API ограничение на выгрузку **1гб за раз**, поэтому за очень длительные промежутки надо будет делать несколько запросов.


In [3]:
"""
    API Отчётов https://appmetrica.yandex.ru/docs/ru/mobile-api/api_v1/intro
"""

class AppMetricaReportAPI():
    def __init__(self, metrics = "ym:ec2:ecomRevenueFiatRUB",
               base_url = "https://api.appmetrica.yandex.ru/stat/v1/data",
               start_date = pendulum.now('Europe/Moscow').subtract(days=1).to_date_string(),
               end_date = 'yesterday',
               dimensions='ym:ec2:ecomOrderId,ym:ec2:regionCity',  # ym:ec2:regionCountry
               filters = "ym:ec2:ecomOrderId!=null AND ym:ec2:ecomOrderId!='null'",
               offset= 1,
               limit= 100000,
               include_undefined = 'true',
               accuracy = 1):

        self.key = "Add your key here, it's empty by default"
        self.application_id = '2398688'
        self.base_url = base_url
        self.start_date = start_date
        self.end_date = end_date    # принимает не только даты, но и ключевые слова "today" и "yesterday"
        self.metrics = metrics
        self.dimensions=dimensions
        self.filters = filters
        self.sort=f'-{self.metrics}'
        self.offset= offset
        self.limit= limit
        self.include_undefined = include_undefined # Включает в ответ строки, для которых значения группировок (group) не определены. Влияет только на первую группировку. По умолчанию выключено.
        self.accuracy = accuracy

    def _is_period_set(self):
        if not hasattr(self,'start_date'):
            self._set_period()

        return True


    def _set_period(self):
        self.start_date = pendulum.now('Europe/Moscow').subtract(days=1).to_date_string()


    def _set_report(self):
        if not hasattr(self,'report_data'):
          self.report()

        return True


    def report(self):

        # функция запросов к метрике
        def appmetrica_ecommerce_request(base_url,application_id: int, key: str, metrics: str, start_date: str,
                          end_date: str, dimensions:str, filters:str, sort:str, offset:int,
                          limit:int, include_undefined:str, accuracy:int):
            
            response = requests.get(
                        url=base_url,
                        headers= {'Authorization': f"OAuth {key}"},
                        params={
                            'id' : application_id,
                            'date1': start_date,
                            'date2': end_date,
                            'metrics': metrics,
                            'dimensions': dimensions,
                            'filters': filters,
                            'sort': sort,
                            'offset': offset,
                            'limit': limit,
                            'include_undefined': include_undefined,
                            'accuracy': accuracy
                        }
                                      )
            self.last_response = response

            return response.json()


        # функция для дозаписи, если первый запрос в основной функции оказался меньше чем всего данных
        def _get_full_responce(num_of_iter):

            data = pd.DataFrame(columns = ['dimensions','metrics'])

            for i in range(1, num_of_iter+1):

                offset_new = self.offset + self.limit * i

                data2 = appmetrica_ecommerce_request(self.base_url, self.application_id, self.key, self.metrics,
                                                self.start_date, self.end_date,
                                                self.dimensions, self.filters, self.sort, offset_new,
                                                self.limit, self.include_undefined,
                                                self.accuracy)
            try:
                data = pd.concat([data, pd.DataFrame(data2['data'])], ignore_index=True)

            except:
                print("Error occurred in _get_full_responce function")
                return(data)

            return(data)



        # приведение датафрейма в порядок
        def _pretty_report_data(data):
            data['transactionId'] = '-'
            data['city'] = '-'

            for j in range(len(data['metrics'][0])):
                data[f'metrics{j}']='-'

            for i in range(len(data)):
                
                data['transactionId'][i] = data['dimensions'][i][0]['name']
                data['city'][i] = data['dimensions'][i][1]['name']

                for j in range(len(data['metrics'][0])):
                    data[f'metrics{j}'][i] = data['metrics'][i][j]

            data.drop(columns=['dimensions','metrics'],inplace=True)

            return data

        # основная функция запроса таблицы отчёта у метрики
        data = appmetrica_ecommerce_request(self.base_url, self.application_id, self.key, self.metrics,
                                            self.start_date, self.end_date,
                                            self.dimensions, self.filters, self.sort, self.offset,
                                            self.limit, self.include_undefined,
                                            self.accuracy)
        # проверка на ошибки
        try:
            
            self.report_data = _pretty_report_data(pd.DataFrame(data['data']))

            if data['total_rows'] > len(self.report_data):

                num_of_iterations = math.ceil((data['total_rows'] - len(self.report_data)) / self.limit)

                additional = _pretty_report_data(_get_full_responce(num_of_iterations))

                self.report_data = pd.concat([self.report_data, additional], ignore_index=True)

        except:
            if not hasattr(self,'report_data'):
                self.report_data = data

            print("Error occurred in report function")



In [10]:
rep = AppMetricaReportAPI(os.getenv("API_KEY"))
rep._set_report()

total_rows 35452


True

In [11]:
data = rep.report_data
data.rename(columns={"metrics0": "ecom_revenue"},inplace=True)

# ClickHouse

In [8]:
# authorization at ClickHouse
HOST= os.getenv("HOST")
PORT= os.getenv("PORT")
USERNAME= os.getenv("USERNAME")
PASSWORD= os.getenv("PASSWORD")

client = clickhouse_connect.get_client(host=HOST, port=PORT, username=USERNAME, password=PASSWORD)

In [15]:
client.insert_df("`temporary`.a_ostrovskaia_appmetrica_transactions_w_city", data)